In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import pandas as pd
%matplotlib inline

In [ ]:
nodes_file = "Department_Collaborate_node.csv"
nodes_df = pd.read_csv(nodes_file, header=None, names=["id", "label"])

In [ ]:
G = nx.Graph()

for _, row in nodes_df.iterrows():
    G.add_node(row['id'], label=row['label'])

In [ ]:
print("Nodes:", G.nodes(data=True))

In [ ]:
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, labels=nx.get_node_attributes(G, 'label'), 
        node_size=700, node_color="lightblue", font_size=10, font_color="black")
plt.show()